In [1]:
import pandas as pd
import requests

In [2]:
columns = ("tweet_text",)
df = pd.DataFrame(columns=columns)

In [3]:
QUERY_BUILDERS = "-has:links -has:mentions lang:pt"
QUERY_LIST = [
    "(samsung OR s22 OR s21 OR s20) (bom OR ótimo OR excelente)",
    "(iphone OR apple) (bom OR ótimo OR excelente)",
    "motorola (bom OR ótimo OR excelente)",
    "xiaomi (bom OR ótimo OR excelente)",
    "(samsung OR s22 OR s21 OR s20) (ruim OR péssimo OR horrível)",
    "(iphone OR apple) (ruim OR péssimo OR horrível)",
    "motorola (ruim OR péssimo OR horrível)",
    "xiaomi (ruim OR péssimo OR horrível)",
    (
        "celular #samsung OR #motorola OR #apple OR #xiaomi OR "
        "#MI OR #iOS OR #LG (bom OR ótimo OR excelente) "
        "(ruim OR péssimo OR horrível)"
    ),
    "(smart tv OR smarttv OR tv) (bom OR ótimo OR excelente)",
    "tv ((bom OR ótimo OR excelente) OR (ruim OR péssimo OR horrível))",
    
]

URL = "https://api.twitter.com/2/tweets/search/recent"


In [4]:
def load_twitter_data(params: dict, headers: dict):
    response = requests.get(
        url=URL,
        params=params,
        headers=headers,
    )
    data = response.json()
    
    return data

In [5]:
idx = 0

for text in QUERY_LIST:
    headers = {
        "Authorization": (
            "Bearer AAAAAAAAAAAAAAAAAAAAACWXjgEAAAAAfLAbTYeJhrL8GjEHfF"
            "%2FyLG2mJzk%3DOtTI7cwVvAE2ijeUA7VrI7OnwFSgntbOG8MdfHKhWHVmZ3U79I"
        )
    }
    params = {
        "query": f"{text} {QUERY_BUILDERS}",
        "max_results": 100,
    }
    
    search_next = True
    
    while search_next:
        twitter_data = load_twitter_data(params, headers)
        
        for item in twitter_data["data"]:
            tweet_data = {
                "tweet_text": item["text"]
            }

            item = pd.DataFrame(tweet_data, index=[idx])
            df = pd.concat([df, item])
            
            idx += 1
        
        if next_token := twitter_data.get("next_token"):
            search_next = True
            params["next_token"] = next_token
        else:
            search_next = False

In [7]:
df.to_csv("./data/twitter_reviews.csv", index=False)

In [10]:
df_loaded = pd.read_csv("./data/twitter_reviews.csv")

In [11]:
df_loaded

,tweet_text
0,Alguém que tenha o Notebook Samsung book me di...
1,O bom de usar Samsung é que desde ontem eu não...
2,atualizei meu samsung pro android 13 bom dia
3,Gente eu passei anos e anos sendo hater da Sam...
4,GENTE ATENÇÃO AQUI RAPIDINHO PFV\n\nalgum de v...
...,...
352,"Alguns dizem que xiaomi é ruim, outros dizem q..."
353,"tô quase voltando pro meu xiaomi, a bateria do..."
354,o face id é algo que me impressiona pra caralh...
355,"meu xiaomi só é boa a bateria mesmo, porque o ..."
